# Ultimate black box

![black-box](black-boxx.png)


In [25]:
# Setup
from dotenv import load_dotenv
from langchain import PromptTemplate, LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

load_dotenv()

gpt35 = ChatOpenAI(
    temperature=0, callbacks=[StreamingStdOutCallbackHandler()], streaming=True
)

In [26]:
# Example of a data connector service

system_template = """You are a data connector service.
Your goal is to transform user record from different schemas into a single unified schema.
Given a user record in arbitrary format, extract the data and transform it to the example schema.
Respond with the JSON of the transformed user record and nothing else.
"""

human_template = """
Input:
{source_user}

Example target user record:
```
{target_example}
```

Transformed user record:
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)


chain = LLMChain(llm=gpt35, prompt=chat_prompt)

target_example = """{
    "user_id": 234,
    "firstName": "Jane",
    "lastName": "Smith",
    "emailAddress": "jane.smith@example.com",
    "mobile": "+11234567890",
    "location": {
        "streetAddress": "456 Elm St",
        "town": "Shelbyville",
        "province": "ON",
        "postalCode": "L5M 3K6",
        "country": "Canada"
    },
    "status": "active",
    "created_at": "2023-06-09T12:30:00Z"
}"""

response = chain.run(
    source_user="""{
    "userID": "A123",
    "fullName": "John Doe",
    "email": "john.doe@example.com",
    "phoneNumber": "123-456-7890",
    "address": {
        "street": "123 Main St",
        "city": "Springfield",
        "state": "IL",
        "zipCode": "62704"
    },
    "accountStatus": "Active",
    "registrationDate": "2018-05-20"
}""",
    target_example=target_example,
)

{
    "user_id": 123,
    "firstName": "John",
    "lastName": "Doe",
    "emailAddress": "john.doe@example.com",
    "mobile": "+11234567890",
    "location": {
        "streetAddress": "123 Main St",
        "town": "Springfield",
        "province": "IL",
        "postalCode": "62704",
        "country": "USA"
    },
    "status": "active",
    "created_at": "2018-05-20T00:00:00Z"
}

In [27]:
response = chain.run(
    source_user="""{
    "userAlias": "tomcat",
    "name": {
        "first": "Tom",
        "last": "Cat"
    },
    "email": "tomcat@example.com",
    "phone": "123-456-7891",
    "addr": {
        "line1": "101 Hollywood Blvd",
        "city": "Los Angeles",
        "state": "CA",
        "postalCode": "90028"
    },
    "signUpDate": "2021-09-15",
    "isVerified": false,
    "preferences": {
        "newsletter": true,
        "notifications": false
    }
}
""",
    target_example=target_example,
)

{
    "user_id": null,
    "firstName": "Tom",
    "lastName": "Cat",
    "emailAddress": "tomcat@example.com",
    "mobile": "123-456-7891",
    "location": {
        "streetAddress": "101 Hollywood Blvd",
        "town": "Los Angeles",
        "province": "CA",
        "postalCode": "90028",
        "country": null
    },
    "status": "inactive",
    "created_at": "2021-09-15T00:00:00Z"
}